In [1]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

# Load the IMDb dataset from a CSV file
df = pd.read_csv('imdb dataset.csv')

# Display the first few rows of the dataset
print(df.head())


   star_rating                     title content_rating   genre  duration  \
0          9.3  The Shawshank Redemption              R   Crime       142   
1          9.2             The Godfather              R   Crime       175   
2          9.1    The Godfather: Part II              R   Crime       200   
3          9.0           The Dark Knight          PG-13  Action       152   
4          8.9              Pulp Fiction              R   Crime       154   

                                         actors_list  
0  [u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt...  
1    [u'Marlon Brando', u'Al Pacino', u'James Caan']  
2  [u'Al Pacino', u'Robert De Niro', u'Robert Duv...  
3  [u'Christian Bale', u'Heath Ledger', u'Aaron E...  
4  [u'John Travolta', u'Uma Thurman', u'Samuel L....  


In [3]:
# Display the first few rows and column names to understand the structure of the DataFrame
print(df.head())
print(df.columns)


   star_rating                     title content_rating   genre  duration  \
0          9.3  The Shawshank Redemption              R   Crime       142   
1          9.2             The Godfather              R   Crime       175   
2          9.1    The Godfather: Part II              R   Crime       200   
3          9.0           The Dark Knight          PG-13  Action       152   
4          8.9              Pulp Fiction              R   Crime       154   

                                         actors_list  
0  [u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt...  
1    [u'Marlon Brando', u'Al Pacino', u'James Caan']  
2  [u'Al Pacino', u'Robert De Niro', u'Robert Duv...  
3  [u'Christian Bale', u'Heath Ledger', u'Aaron E...  
4  [u'John Travolta', u'Uma Thurman', u'Samuel L....  
Index(['star_rating', 'title', 'content_rating', 'genre', 'duration',
       'actors_list'],
      dtype='object')


In [4]:
#Problem 1: Word2Vec Processing with Classifier

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize


# Tokenize the titles
nltk.download('punkt')
df['tokenized_text'] = df['title'].apply(lambda x: word_tokenize(x.lower()))


df['label'] = df['genre'].apply(lambda x: 1 if x == 'Crime' else 0)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train Word2Vec model on tokenized titles
model = Word2Vec(sentences=df['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)

# Convert titles to vectors using Word2Vec
def get_vector(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

df['word2vec_vector'] = df['tokenized_text'].apply(lambda x: get_vector(x, model))

# Prepare features and labels
X = np.array(df['word2vec_vector'].tolist())
y = df['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict and evaluate the classifier
y_pred = classifier.predict(X_test)
print(f'Word2Vec Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


Word2Vec Accuracy: 0.9030612244897959
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       177
           1       0.00      0.00      0.00        19

    accuracy                           0.90       196
   macro avg       0.45      0.50      0.47       196
weighted avg       0.82      0.90      0.86       196



C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
# Problem 2: Implement GloVe Embeddings and Compare Results
import numpy as np

# Path to your GloVe embeddings file
glove_file_path = 'glove.6B.100d.txt'

# Load GloVe embeddings into a dictionary
glove_embeddings = {}
with open(glove_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

print(f"Loaded {len(glove_embeddings)} word vectors.")



Loaded 400000 word vectors.


In [7]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Assuming your dataframe is already loaded as df
nltk.download('punkt')

# Tokenize the titles
df['tokenized_text'] = df['title'].apply(lambda x: word_tokenize(x.lower()))

# Convert titles to GloVe vectors
def get_glove_vector(sentence, glove_embeddings):
    vectors = [glove_embeddings.get(word, np.zeros(100)) for word in sentence]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)

df['glove_vector'] = df['tokenized_text'].apply(lambda x: get_glove_vector(x, glove_embeddings))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Prepare features (GloVe vectors) and labels
X = np.array(df['glove_vector'].tolist())
y = df['genre'].apply(lambda x: 1 if x == 'Crime' else 0).values  # Example: classify 'Crime' vs. others

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict and evaluate the classifier
y_pred = classifier.predict(X_test)
print(f'GloVe Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


GloVe Accuracy: 0.8724489795918368
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       177
           1       0.12      0.05      0.07        19

    accuracy                           0.87       196
   macro avg       0.51      0.51      0.50       196
weighted avg       0.83      0.87      0.85       196



In [9]:
pip install transformers torch


Note: you may need to restart the kernel to use updated packages.


In [12]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Tokenize the dataset using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the titles
train_texts, test_texts, train_labels, test_labels = train_test_split(df['title'], df['label'], test_size=0.2, random_state=42)

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Step 2: Create a PyTorch dataset class for IMDb data
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, list(train_labels))
test_dataset = IMDbDataset(test_encodings, list(test_labels))

# Step 3: Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Step 4: Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"      # Evaluate every epoch
)

# Step 5: Create the Trainer object
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

# Step 6: Train the model
trainer.train()

# Step 7: Evaluate the model
results = trainer.evaluate()

# Step 8: Print the accuracy
print(f'BERT Accuracy: {results["eval_accuracy"]}')
print("Classification Report:")
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)
print(classification_report(test_labels, predictions))


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [11]:
pip install accelerate -U


                                              0.0/315.1 kB ? eta -:--:--
     --------------------                   174.1/315.1 kB 5.3 MB/s eta 0:00:01
     -------------------------------------  307.2/315.1 kB 4.8 MB/s eta 0:00:01
     -------------------------------------- 315.1/315.1 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Step 1: Load and Prepare a Smaller Subset of Data
# Assuming df is already loaded with your IMDb dataset
df = pd.read_csv('imdb dataset.csv')

# Trim the dataset to 100 samples for quick training (you can adjust this number)
df_small = df.sample(100, random_state=42)

# Create binary labels: 1 for 'Crime', 0 for other genres
df_small['label'] = df_small['genre'].apply(lambda x: 1 if x == 'Crime' else 0)

# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df_small['title'], df_small['label'], test_size=0.2, random_state=42)

# Step 2: Tokenize the Texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128  # Max length of text sequences

class IMDbDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Prepare DataLoader
train_dataset = IMDbDataset(train_texts.tolist(), train_labels.tolist(), tokenizer, max_len)
test_dataset = IMDbDataset(test_texts.tolist(), test_labels.tolist(), tokenizer, max_len)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # Smaller batch size for faster training
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Step 3: Initialize Model, Optimizer, and Loss Function
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss().to(device)

# Step 4: Training Loop (Single Epoch for Simplicity)
def train(model, data_loader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        _, preds = torch.max(outputs.logits, dim=1)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct_predictions += torch.sum(preds == labels)
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions.double() / len(data_loader.dataset)
    
    print(f"Training Loss: {avg_loss}, Training Accuracy: {accuracy}")
    return avg_loss, accuracy

# Step 5: Evaluation Function
def evaluate(model, data_loader, loss_fn, device):
    model.eval()
    correct_predictions = 0
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
    
    accuracy = correct_predictions.double() / len(data_loader.dataset)
    print(f"Validation Accuracy: {accuracy}")
    return accuracy

# Train for 1 Epoch
train(model, train_loader, loss_fn, optimizer, device)

# Evaluate on Test Set
evaluate(model, test_loader, loss_fn, device)

# Step 6: Generate Classification Report
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("Classification Report:")
print(classification_report(true_labels, predictions))


C:\Users\HP\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please u

Training Loss: 1.0000064194202423, Training Accuracy: 0.125
Validation Accuracy: 0.2
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.11      0.20        18
           1       0.11      1.00      0.20         2

    accuracy                           0.20        20
   macro avg       0.56      0.56      0.20        20
weighted avg       0.91      0.20      0.20        20

